In [1]:
import pandas as pd
import lightgbm as lgb

import matplotlib.pyplot as plt

%matplotlib inline

try:
    # To enable interactive mode you should install ipywidgets
    # https://github.com/jupyter-widgets/ipywidgets
    from ipywidgets import interact, SelectMultiple
    INTERACTIVE = True
except ImportError:
    INTERACTIVE = False

In [2]:
data = pd.read_csv('D:/ruanjian/streetview/Image Segmentation/PSPNet-tensorflow-master/csvv/NKDE_pedeshot.csv')

#create a python list of feature names
feature_cols = ['SKY','VEGETATION','BUILDING','ROAD','SIDEWALK','VEHICLE','streetligh','bicycle','car','fence','minibike','sidewalk1','trade_name','luojia_100m','person']
# use the list to select a subset of the original DataFrame
X = data[feature_cols]
# equivalent command to do this in one line
X = data[['SKY','VEGETATION','BUILDING','ROAD','SIDEWALK','VEHICLE','streetligh','bicycle','car','fence','minibike','sidewalk1','trade_name','luojia_100m','person']]
# print the first 5 rows
print (X.head())
# check the type and shape of X
print (type(X))
print (X.shape)
y = data['pedes_all']
from sklearn.model_selection import train_test_split  #这里是引用了交叉验证
X_train,X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print (X_train.shape)
print (y_train.shape)
print (X_test.shape)
print (y_test.shape)

#y_train = df_train[0]
#y_test = df_test[0]
#X_train = df_train.drop(0, axis=1)
#X_test = df_test.drop(0, axis=1)

        SKY  VEGETATION  BUILDING      ROAD  SIDEWALK   VEHICLE  streetligh  \
0  0.487926    0.023288  0.229259  0.209446  0.013710  0.014668    0.000148   
1  0.469794    0.052732  0.242180  0.109091  0.019171  0.056696    0.001113   
2  0.394408    0.062216  0.329109  0.171277  0.012435  0.014871    0.000655   
3  0.151914    0.548184  0.083486  0.118386  0.013062  0.031127    0.003173   
4  0.186834    0.383685  0.199099  0.111971  0.038124  0.021678    0.001320   

    bicycle       car     fence  minibike  sidewalk1  trade_name  luojia_100m  \
0  0.002301  0.014668  0.000061  0.000000   0.013710    0.000000     0.004002   
1  0.006835  0.054895  0.018103  0.000000   0.019171    0.000055     0.001633   
2  0.002497  0.014849  0.004650  0.000000   0.012435    0.000000     0.001633   
3  0.003182  0.030306  0.000000  0.017857   0.013062    0.000077     0.002669   
4  0.019976  0.012429  0.000002  0.002089   0.038124    0.004881     0.002669   

     person  
0  0.011401  
1  0.00261

In [3]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_test = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [4]:
params = {
    'num_leaves': 10,
    'learning_rate': 0.1,
    'metric': ['l1', 'l2'],
    'feature_fraction': 0.9, # 建树的特征选择比例
    'bagging_fraction': 0.8, # 建树的样本采样比例
    'bagging_freq': 5,  # k 意味着每 k 次迭代执行bagging
    'verbose': -1
}

In [14]:
evals_result = {}  # to record eval results for plotting
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=100,
                valid_sets=[lgb_train, lgb_test],
                feature_name=[feature_cols[i] for i in range(X_train.shape[-1])],
                categorical_feature=[21],
                evals_result=evals_result,
                verbose_eval=10)

[10]	training's l1: 0.210151	training's l2: 0.0908118	valid_1's l1: 0.227191	valid_1's l2: 0.106746
[20]	training's l1: 0.177577	training's l2: 0.0763777	valid_1's l1: 0.203413	valid_1's l2: 0.0994272
[30]	training's l1: 0.165501	training's l2: 0.0706525	valid_1's l1: 0.195961	valid_1's l2: 0.0973393
[40]	training's l1: 0.159151	training's l2: 0.0661994	valid_1's l1: 0.194285	valid_1's l2: 0.0973197
[50]	training's l1: 0.154886	training's l2: 0.0627826	valid_1's l1: 0.1946	valid_1's l2: 0.097328
[60]	training's l1: 0.149993	training's l2: 0.0589688	valid_1's l1: 0.193015	valid_1's l2: 0.0958045
[70]	training's l1: 0.147881	training's l2: 0.0566311	valid_1's l1: 0.194135	valid_1's l2: 0.0959612
[80]	training's l1: 0.145897	training's l2: 0.054662	valid_1's l1: 0.194136	valid_1's l2: 0.0960363
[90]	training's l1: 0.142305	training's l2: 0.0522133	valid_1's l1: 0.194805	valid_1's l2: 0.0964546
[100]	training's l1: 0.139642	training's l2: 0.0502674	valid_1's l1: 0.19509	valid_1's l2: 0.096

In [25]:
gbm.save_model('Regressionmodel.txt')

In [35]:
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

In [30]:
print(list(y_pred))

[0.16806226173426442, -0.0012894627641018997, 0.41488080080046014, 0.09027556487213284, 0.022519148218727893, 0.258116584201034, -0.006305997999646624, 0.14254049781028624, 0.0538605409615074, -0.01027528355614035, -0.009952507076496085, 0.4207206174109534, 0.2765224967365114, 0.8767272990158792, 0.12452247180076964, -0.011237410843562683, 0.09712768991640829, 0.03229681725623327, -0.01753373795349153, 0.6384967098660899, 0.4741445056103047, 0.02460463889651622, 0.02244134418254, 0.33361810503285166, 0.5733237822523729, 0.29385441526189227, -0.018883102621578468, -0.012043552361221921, 0.2842374934305086, 0.3169655665587109, 0.0674069068300153, 0.06346699559628277, 0.3464410475125132, 0.03242592371438174, -0.03383153368191863, 0.11592032756430062, 0.05143367523449754, -0.012629258501345646, -0.002270187289653526, -0.02667946230324275, 0.14985677041888135, 0.06591289268704298, 0.49819531557720215, 0.1185159866574052, 0.017584626458767783, 0.02639050410777557, 0.02235309393007088, 0.5409

evals_result = {}  # to record eval results for plotting
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=100,
                valid_sets=[lgb_train, lgb_test],
                feature_name=['f' + str(i + 1) for i in range(X_train.shape[-1])],
                categorical_feature=[21],
                evals_result=evals_result,
                verbose_eval=10)

In [33]:
import numpy as np
np.savetxt("y_pred.csv", y_pred, delimiter=",")

In [15]:
def render_metric(metric_name):
    ax = lgb.plot_metric(evals_result, metric=metric_name, figsize=(10, 5))
    plt.show()

In [16]:
if INTERACTIVE:
    # create widget to switch between metrics
    interact(render_metric, metric_name=params['metric'])
else:
    render_metric(params['metric'][0])

interactive(children=(Dropdown(description='metric_name', options=('l1', 'l2'), value='l1'), Output()), _dom_c…

In [17]:
def render_plot_importance(importance_type, max_features=10,
                           ignore_zero=True, precision=3):
    ax = lgb.plot_importance(gbm, importance_type=importance_type,
                             max_num_features=max_features,
                             ignore_zero=ignore_zero, figsize=(12, 8),
                             precision=precision)
    plt.show()

In [18]:
if INTERACTIVE:
    # create widget for interactive feature importance plot
    interact(render_plot_importance,
             importance_type=['split', 'gain'],
             max_features=(1, X_train.shape[-1]),
             precision=(0, 10))
else:
    render_plot_importance(importance_type='split')
    
#'Split' contains numbers of times the feature is used in a model.
#'Gain' esult contains total gains of splits which use the feature.

interactive(children=(Dropdown(description='importance_type', options=('split', 'gain'), value='split'), IntSl…

In [20]:
def render_histogram(feature):
    ax = lgb.plot_split_value_histogram(gbm, feature=feature,
                                        bins='auto', figsize=(10, 5))
    plt.show()

In [21]:
if INTERACTIVE:
    # create widget for interactive split value histogram
    interact(render_histogram,
             feature=gbm.feature_name())
else:
    render_histogram(feature='f26')

interactive(children=(Dropdown(description='feature', options=('SKY', 'VEGETATION', 'BUILDING', 'ROAD', 'SIDEW…

In [23]:
def render_tree(tree_index, show_info, precision=3):
    show_info = None if 'None' in show_info else show_info
    return lgb.create_tree_digraph(gbm, tree_index=tree_index,
                                   show_info=show_info, precision=precision)

In [24]:
if INTERACTIVE:
    # create widget to switch between trees and control info in nodes
    interact(render_tree,
             tree_index=(0, gbm.num_trees() - 1),
             show_info=SelectMultiple(  # allow multiple values to be selected
                 options=['None',
                          'split_gain',
                          'internal_value',
                          'internal_count',
                          'internal_weight',
                          'leaf_count',
                          'leaf_weight',
                          'data_percentage'],
                 value=['None']),
             precision=(0, 10))
    tree = None
else:
    tree = render_tree(53, ['None'])
tree

interactive(children=(IntSlider(value=49, description='tree_index', max=99), SelectMultiple(description='show_…